In [19]:
#Imports:
from ncps.wirings import AutoNCP
from ncps.torch import CfC
import pytorch_lightning as pl
from pytorch_lightning.loggers import CSVLogger
from numpy import genfromtxt
import numpy as np
import torch
import torch.utils.data as data
import matplotlib as plt
import torch.nn as nn

In [ ]:
#PROPER FORMATTING
#Time sequences are 10 timepoints (Messages) with 7 features per message.
#Organized by car.

#Current Simulation File
dataFile = 'Data/CfCMultiExtension/DoS_0709.csv'

dataSet = genfromtxt(dataFile, delimiter=',')
batchSize = 64
# Ceate dataloader and fill with (BSM, attk#). Expanding to add 0th dimension for batches.
# Batch size should be 64 for the low density simulations and 128 for high density simulations.
# No shuffle to keep batches on same vehicle.
# Num_workers is set to = num CPU cores
dataSet[0:-1,:] = dataSet[1:,:] # Get rid of the first null value of the dataset
print(dataSet.shape)
# count subsets per vehicle
unq, counts = np.unique(dataSet[:, 2], return_counts = True)
# Create new arrays per vehicle for federated learning
splits = np.split(dataSet, np.cumsum(counts)[:-1])
sender = 0
lastSenderCount = 0
newData = []
# Organize dataset into sets of 10 messages by sender
while sender < counts.shape[0]:
    # Loop through sender
    index = 0
    while index < counts[sender] - 10:
        # Loop through messages from sender
        newData.append(dataSet[lastSenderCount+index:lastSenderCount +index+10])
        index += 5
    sender += 1
    lastSenderCount = counts[sender-1]
dataSet = torch.tensor(newData)
len = dataSet.shape[0]
trainPerc = 80
# Create seperate datasets for testing and training, using Train Percentage as metric for split
trainDataIn = torch.Tensor(dataSet[:int(len*(trainPerc/100)),:,1:10]).float()
trainDataOut = torch.Tensor(np.int_(dataSet[:int(len*(trainPerc/100)),:,11])).long()
testDataIn = torch.Tensor(dataSet[int(len*(trainPerc/100)):,:,1:10]).float()
testDataOut = torch.Tensor(np.int_(dataSet[int(len*(trainPerc/100)):,:,11])).long()
newsetIn = []
newsetOut = []
# Create dataset of 1/100th of the entries for quicker testing during development
for index in range(0,int(len * (trainPerc/100))):
    if not (int(index/10) % 100):
        newsetIn.append(dataSet[index,:,1:10])
        newsetOut.append((dataSet[index,:,11]))
testingIn = torch.Tensor(np.array(newsetIn)).float()
testingOut = torch.Tensor(np.array(newsetOut)).long()
# Create Dataloaders for all the datasets
dataLoaderTrain = data.DataLoader(data.TensorDataset(trainDataIn, trainDataOut), batch_size=batchSize, shuffle=False, num_workers=16)
dataLoaderTest = data.DataLoader(data.TensorDataset(testDataIn, testDataOut), batch_size=batchSize, shuffle=False, num_workers=16)
testingDataLoader = data.DataLoader(data.TensorDataset(testingIn, testingOut), batch_size=batchSize, shuffle = False, num_workers=16)

(4957201, 12)


In [87]:
print(trainDataIn.shape)
print(trainDataOut.shape)
print(testingOut.shape)
print(testingIn.shape)
print(trainDataIn[-1])
print(trainDataOut[1])
print(testDataIn[15])
print(testingOut[0])
print(dataSet[0:2])
print(dataSet[90].shape)


torch.Size([3965760, 9])
torch.Size([3965760])
torch.Size([39660])
torch.Size([39660, 9])
tensor([1.9347e+04, 1.9611e+04, 3.1161e+04, 6.6291e+01, 2.4601e+02, 3.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00])
tensor(0)
tensor([1.9347e+04, 1.9611e+04, 3.1161e+04, 6.6291e+01, 2.4601e+02, 4.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00])
tensor(0)
[[0.00000000e+00 4.50000000e+01 9.00000000e+00 2.52126029e+04
  7.93602911e+00 9.57097011e+01 1.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.00000000e+00 4.50000000e+01 9.00000000e+00 2.52136029e+04
  1.32969123e+01 1.06405701e+02 1.00000000e+00 2.28293295e+00
  1.43427328e+01 2.66527912e-01 5.15145214e-01 0.00000000e+00]]
(12,)


In [ ]:
# Creating Learner
class CfCLearner(pl.LightningModule):
    def __init__(self, model, lr):
        super().__init__()
        self.model = model
        self.lr = lr
        self.lossFunc = nn.CrossEntropyLoss()

    def training_step(self, batch, batch_idx):
        # Get in and out from batch
        inputs, target = batch
        # Put input through model
        output, _ = self.model.forward(inputs)
        # Reorganize inputs for use with loss function
        output = output.permute(0, 2, 1)
        # Calculate Loss using Cross Entropy Loss 
        loss = self.lossFunc(output, target)
        self.log("trainLoss", loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        # Get in and out from batch
        inputs, target = batch
        # Put input through model
        output, _ = self.model.forward(inputs)
        # Reorganize inputs for use with loss function
        output = output.permute(0, 2, 1)
        print(f"output: {output.shape}")
        print(f"target: {target.shape}")
        # Calculate Loss using Cross Entropy Loss 
        loss = self.lossFunc(output, target)
        self.log("valLoss", loss, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        return self.validation_step(batch, batch_idx)

    def configure_optimizers(self):
        # Using AdamW optomizer based on info from paper
        return torch.optim.AdamW(self.model.parameters(), lr = self.lr)

In [23]:
class Modena(nn.Module): 
    # CfC with feed-forward layer to classify at end.
    def __init__(self, inputSize, unitNum, motorNum, outputDim, batchFirst = True):
        super().__init__()
        # Create NCP wiring for CfC
        wiring = AutoNCP(unitNum, motorNum)
        # Create CfC model with inputs and wiring
        self.cfc = CfC(inputSize, wiring, batch_first=batchFirst)
        # Create feed-forward layer
        self.fF = nn.Linear(motorNum, outputDim)
    
    def forward(self, batch, hidden = None):
        batch, hidden = self.cfc(batch, hidden) # Pass inputs through CfC
        out = nn.functional.relu(self.fF(batch)) # pass through FeedForward Layer, then make 0 minimum
        return out, hidden # Return the guess and the hidden state

In [ ]:
# Function to run model through a testing dataset and calculate accuracy. Can be expanded to give more metrics and more useful metrics.
def test(model, dataIn, dataOut):
    # Put input data through model and determine classification
    with torch.no_grad():
        outs = np.asarray(model(dataIn)[0])
    outs = torch.from_numpy(outs)
    # Get the label with the maximum confidence for determining classification
    _, res = torch.max(outs, 2)
    countR = 0
    numZero = 0
    tot = outs.shape[0]
    total = 0
    for i in range(0, tot):
        # Loop through sequences of 10 each
        for t in range(0, res[i].shape[0]):
            # Loop through the sub-sequences
            if res[i,t] == dataOut[i,t]:
                # Check if label is correct, and add to count right accordingly
                countR += 1
            if dataOut[i,t] == 0:
                # If the label is zero, increment the count of zeroes to determine if model is just outputting zeroes
                numZero += 1
            total += 1
    # Calculate percent correct and percent zero
    perc = (countR/total) * 100
    percZero = (numZero/total) * 100
    print("Model got " + str(countR) + "/" + str(total) + " right. Accuracy of " + str(perc) + "%")
    print(str(percZero) + "% Zeroes.")
    return countR, total, perc, percZero

In [25]:
#20 units, 8 motor neuron (output)

#input size = 9, 20 units, 8 motor neurons, and 20 possible outputs.
model = Modena(9, 20, 8, 20)
learner = CfCLearner(model, lr=0.001) #Tune units, lr
trainer = pl.Trainer(
    logger = CSVLogger('log'), # Set ouput destination of logs, logging accuracy every 50 steps
    max_epochs= 10, # Number of epochs to train for
    gradient_clip_val= 1 # This is said to stabilize training, but we should test if that is true
    )

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [25]:
learner.validation_step(next(iter(dataLoaderTest)), 0)

output: torch.Size([64, 10, 20])
target: torch.Size([64, 10])
output: torch.Size([64, 20, 10])


c:\Users\will\miniconda3\envs\Kettering\Lib\site-packages\pytorch_lightning\core\module.py:441: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`


tensor(2.9553, grad_fn=<NllLoss2DBackward0>)

In [26]:
print("Before Training:")
test(model, testDataIn, testDataOut)

Before Training:
Model got 0/1966620 right. Accuracy of 0.0%
39.65916140382993% Zeroes.


(0, 1966620, 0.0, 39.65916140382993)

In [27]:
trainer.fit(learner, testingDataLoader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type             | Params | Mode 
------------------------------------------------------
0 | model    | Modena           | 1.7 K  | train
1 | lossFunc | CrossEntropyLoss | 0      | train
------------------------------------------------------
1.4 K     Trainable params
280       Non-trainable params
1.7 K     Total params
0.007     Total estimated model params size (MB)
27        Modules in train mode
0         Modules in eval mode


Epoch 9: 100%|██████████| 123/123 [00:22<00:00,  5.52it/s, v_num=26, trainLoss=0.257]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 123/123 [00:22<00:00,  5.51it/s, v_num=26, trainLoss=0.257]


In [28]:
print("After Training:")
countR, tot, perc, percZero = test(model, testDataIn, testDataOut)

After Training:
Model got 1666292/1966620 right. Accuracy of 84.72872237646317%
39.65916140382993% Zeroes.
